Беспалова Ульяна
Группа ML-11

Тот же файлик на Гитхабе: 

In [1]:
from google.colab import drive
drive.mount("/content/drive/")

ModuleNotFoundError: No module named 'google'

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('seaborn-talk')
plt.rcParams['figure.figsize'] = (15,10)

# Задача про датчики на руках

В этом задании мы применим метод главных компонент на многомерных данных и постараемся найти оптимальную размерность признаков для решения задачи классификации

Исходными [данными](http://archive.ics.uci.edu/ml/machine-learning-databases/auslan2-mld/auslan.data.html) являются показания различных сенсоров, установленных на руках человека, который умеет общаться на языке жестов.

В данном случае задача ставится следующим образом: по показаниям датчиков (по 11 сенсоров на каждую руку) определить слово, которое было показано человеком.

Как можно решать такую задачу?

Показания датчиков представляются в виде временных рядов. Посмотрим на показания для одного из "слов"

In [ ]:
# Загружаем данные сенсоров
df_database = pd.read_csv('./drive/MyDrive/Colab_Notebooks/Technopark_ML/dz_4/data/sign_database.csv')

# Загружаем метки классов
sign_classes = pd.read_csv('./drive/MyDrive/Colab_Notebooks/Technopark_ML/dz_4/data/sign_classes.csv', index_col=0, header=0, names=['id', 'class'])

In [ ]:
# Столбец id - идентификаторы "слов"
# Столбец time - метка времени
# Остальные столбцы - показания серсоров для слова id в момент времени time

df_database.head()

In [ ]:
sign_classes.head()

In [ ]:
# Выберем одно из слов с идентификатором = 0
sign0 = df_database.query('id == 0')\
                   .drop(['id'], axis=1)\
                   .set_index('time')

In [ ]:
sign0.plot()

Для каждого из "слов" у нас есть набор показаний сенсоров с разных частей руки в каждый момент времени.

Идея нашего подхода будет заключаться в следующем – давайте для каждого сенсора составим набор характеристик (например, разброс значений, максимальное, минимальное, среднее значение, количество "пиков", и т.п.) и будем использовать эти новые "признаки" для решения задачи классификации.

## Расчет новых признаков

Признаки мы будем считать с помощью библиотеки [tsfresh](http://tsfresh.readthedocs.io/en/latest/index.html). Генерация новых признаков может занять много времени, поэтому мы сохранили посчитанные данные, но при желании вы можете повторить вычисления.

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Если не хотите долго ждать - не убирайте комментарии
# !pip install tsfresh


# import tsfresh
# from tsfresh.feature_extraction import extract_features
# from tsfresh.feature_selection import select_features
# from tsfresh.utilities.dataframe_functions import impute
# from tsfresh.feature_extraction import ComprehensiveFCParameters, MinimalFCParameters, settings, EfficientFCParameters


# sign_features = extract_features(df_database, column_id='id', column_sort='time',
#                                  default_fc_parameters=EfficientFCParameters(),
#                                  impute_function=impute)

# enc = LabelEncoder()
# enc.fit(sign_classes.loc[:, 'class'])
# sign_classes.loc[:, 'target'] = enc.transform(sign_classes.loc[:, 'class'])

# sign_features_filtered = select_features(sign_features, sign_classes.loc[:, 'target'])

# filepath = './drive/MyDrive/Colab_Notebooks/Technopark_ML/dz_4/data/tsfresh_features_filt.csv.gz'
# sign_features_filtered.to_csv(filepath, compression='gzip')

In [ ]:
# !cd ./drive/MyDrive/Colab_Notebooks/Technopark_ML/dz_4/data
# !wget https://www.dropbox.com/s/x6b9mqxlw5ijcuf/tsfresh_features_filt.csv.gz?dl=0 -O ./drive/MyDrive/Colab_Notebooks/Technopark_ML/dz_4/data/tsfresh_features_filt.csv.gz

In [ ]:
filepath = './drive/MyDrive/Colab_Notebooks/Technopark_ML/dz_4/data/tsfresh_features_filt.csv.gz'
sign_features_filtered = pd.read_csv(filepath)

In [ ]:
sign_features_filtered.shape

In [ ]:
sign_features_filtered.head()

## Базовая модель

В результате у нас получилось очень много признаков (11516 или больше), давайте применим метод главных компонент, чтобы получить сжатое признаковое представление, сохранив при этом предиктивную силу в модели.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Создадим бейзлайн без уменьшения размерности. Гиперпараметры модели подбирались произвольно

In [ ]:
from sklearn import preprocessing
enc = preprocessing.LabelEncoder()
enc.fit(sign_classes.loc[:, 'class'])
sign_classes.loc[:, 'target'] = enc.transform(sign_classes.loc[:, 'class'])

In [ ]:
# Подготовим данные на вход в модель

# признаки
X = sign_features_filtered.values

# классы
y = sign_classes.target.values

In [ ]:
# Будем делать кросс-валидацию на 5 фолдов
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)

base_model = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', KNeighborsClassifier(n_neighbors=9))
])

base_cv_scores = cross_val_score(base_model, X, y, cv=cv, scoring='accuracy')

In [ ]:
base_cv_scores.mean()

## Метод главных компонент (5 баллов)

* (1) Добавьте в пайплайн `base_model` шаг с методом главных компонент. Начиная с версии 0.18 в sklearn добавили разные солверы для PCA. Дополнитенльно задайте в модели следующие параметры: `svd_solder = "randomized"` и `random_state=123`. Остальные гиперпараметры модели и способ кросс-валидации оставьте без изменений
* (1 + 2) Найдите такое наименьшее количество главных компонент, что качество нового пайплайна превыcит 90%. За +2 бала найдите решение не полным перебором (руками перебрать пару значений, тоже считается полным перебором). Можете считать, что функция зависимости качества от количества компонент монотонно неубывающая. 
* (1) Укажите долю объяснённой дисперсии при найденной настройке PCA  (для этого надо обучить PCA на всех данных). Долю объясненной дисперсии можно найти в атрибуте `explained_variance_ratio`

In [ ]:
# 1) Произвольное число компонент

base_model_pca = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(svd_solver="randomized", random_state=123, n_components=10)),
    ('clf', KNeighborsClassifier(n_neighbors=9))
])

base_cv_scores = cross_val_score(base_model_pca, X, y, cv=cv, scoring='accuracy')

print('n_components=10: ', base_cv_scores.mean())

In [ ]:
# 2) Найдем минимальное n_components, для которого результат будет более 90%

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
ss = StandardScaler()
X_scaled = ss.fit_transform(X)

In [ ]:
pca = PCA(n_components=200)
pca.fit(X_scaled)

In [ ]:
sns.set(style='whitegrid')
fig, pl = plt.subplots()
pl.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('n_components')
plt.ylabel('Объясненная дисперсия')

plt.axvline(linewidth=2, color='violet', linestyle = '--', x=80, ymin=0, ymax=1)
fig.set_figwidth(8)
fig.set_figheight(6)
display(plt.show())

# На графике видим, что объясненная дисперсия перестает значительно расти при n_components≈80. 

In [ ]:
base_model_pca = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(svd_solver="randomized", random_state=123, n_components=80)),
    ('clf', KNeighborsClassifier(n_neighbors=9))
])

base_cv_scores = cross_val_score(base_model_pca, X, y, cv=cv, scoring='accuracy')

print('n_components=80: ', base_cv_scores.mean())

In [ ]:
evr = pca.explained_variance_ratio_
cvr = np.cumsum(pca.explained_variance_ratio_)
pca_df = pd.DataFrame()
pca_df['Cumulative Variance Ratio'] = cvr
pca_df['Explained Variance Ratio'] = evr

def _color(val):
  color = 'blue' if 0.623<val and val<0.626 else ''
  return 'color: %s' % color 
pca_df[75:85].style.applymap(_color)

In [ ]:
# 3) Доля объяснённой дисперсии при n_components=80 составляет 0.624504

# Задача про кластеризацию текстов

Рассмотрим коллекцию новостных сообщений за первую половину 2017 года. Про каждое новостное сообщение известны:
* его заголовок и текст
* дата его публикации
* событие, о котором это новостное сообщение написано 
* его рубрика 

In [ ]:
df = pd.read_csv('./drive/MyDrive/Colab_Notebooks/Technopark_ML/dz_4/data/news.csv', encoding='utf8')
df.head()

In [ ]:
df.loc[:, 'class'].value_counts()

In [ ]:
true_label = df.loc[:, 'class']

Попробуем кластеризовать документы (каким-либо методом) и сравним полученное разбиение с данными рубликами с помощью ARI

## Стандартная предобработка текстов

Ниже выполняется набор операций по предобработке текстов. Возможно на этом курсе про это не будет идти речь, однако идея довольно простая. Просто следите за комментариями =)

In [ ]:
df.text[0] # Так выглядит первая статья до преобработки

In [ ]:
import re

# Оставляем только кириллические символы
regex = re.compile(u"[А-Яа-я]+")

def words_only(text, regex=regex):
    return " ".join(regex.findall(text))

In [ ]:
df.text = df.text.str.lower()
df.loc[:, 'text'] = df.text.apply(words_only)

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords

# Удаляем стоп-слова
mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', u'также',  'т', 'д', '-', '-']

def  remove_stopwords(text, mystopwords = mystopwords):
    try:
        return u" ".join([token for token in text.split() if not token in mystopwords])
    except:
        return u""

In [ ]:
df.text[0]

In [ ]:
# !pip install pymystem3
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!mv ./mystem ./drive/MyDrive/Colab_Notebooks/Technopark_ML/dz_4

In [ ]:
!sudo chmod 777 ./drive/MyDrive/Colab_Notebooks/Technopark_ML/dz_4/mystem

In [ ]:
%%time 
from pymystem3 import Mystem

# нормализуем текст
bin_path = "./drive/MyDrive/Colab_Notebooks/Technopark_ML/dz_4/mystem"
m = Mystem(bin_path)
def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " ! "

In [ ]:
df.text = df.text.apply(lemmatize)

In [ ]:
mystoplemmas = [u'который', u'прошлый', u'сей', u'свой', u'наш', u'мочь']

# Еще кое-что удаляем
def  remove_stoplemmas(text, mystoplemmas = mystoplemmas):
    try:
        return " ".join([token for token in text.split() if not token in mystoplemmas])
    except:
        return ""

df.text = df.text.apply(remove_stoplemmas)  

In [ ]:
df.text[3] # так выгладит третья статья после предобработки

## Вычисление сходства (1 балл)

С помощью `TfidfVectorizer` и `pairwise_distances` расчитайте **косинусное** расстояние между всеми парами документов к корпусе.

Про векторизацию текстов с помощью `CountVectorizer` или `TfidfVectorizer` можно почитать [тут](https://www.machinelearningmastery.ru/gentle-introduction-bag-words-model/) 

Запишите результат в переменную `S`

In [ ]:
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances

In [ ]:
vect = TfidfVectorizer()
texts = df.text.values

In [ ]:
len(texts)

In [ ]:
matrix = vect.fit_transform(texts)

vecs = np.asarray(matrix.todense())
vecs.shape

In [ ]:
S = pairwise_distances(vecs, metric='cosine')
S.shape

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(data=S, cmap = 'Spectral').set(xticklabels=[],yticklabels=[])


Вы должны пронаблюдать, что между некоторыми текстами есть довольно высокое сходство по содержанию слов - мы видим, что вдоль диагонали матрицы попарых расстояний есть "блоки" по похожести. 

Нам конечно же повезло, так как тексты заранее были удачно упорядочены. В реальности такой живописной картины мы скорее всего не увидим.

Попробуем их кластеризовать

## DBSCAN (4 балла)
* (2) Воспользуйтесь методикой оценки параметров для алгоритма DBSCAN. Не копипастите `min_pts = 2` из семинара! Используйте **косинусную** меру близости.
* (1) Выделите кластеры. Для каждого кластера (кроме -1, если он будет) выведите несколько текстов и умозрительно определите его тематику. Можете подсмотреть в исходные тематики корпуса
* (1) Оцените сходство с изначальными рубриками визуально (с помощью матрицы перемешивания) и с помощью [Adjusted Rand Index](https://en.wikipedia.org/wiki/Rand_index)

In [ ]:
# 1) Согласно карте схожести, размер самого маленького "блока" похожих элементов составляет 10-15 штучек
# Примем min_pts = 10

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN

In [ ]:
# Определим наиболее подходящее значение eps 

min_pts = 10
nn = NearestNeighbors(n_neighbors = min_pts, metric='cosine')
nn.fit(vecs)

In [ ]:
dist, _ = nn.kneighbors(vecs)

In [ ]:
dist_last = dist[:, -1]
dist_last = np.sort(dist_last)

In [ ]:
plt.figure(figsize = (8,5))
plt.plot(dist_last)

In [ ]:
# Примем eps = 0.6

In [ ]:
clustering = DBSCAN(eps = 0.6, min_samples=min_pts, metric='cosine').fit(vecs)

In [ ]:
labels = clustering.labels_
len(set(labels))

In [ ]:
# 2) Попробуем интерпретировать полученные кластеры

In [ ]:
df_clasters = pd.concat([df, pd.DataFrame(labels, columns=["dbscan_labels"])], axis=1)

In [ ]:
# label 0
df_clasters[df_clasters.dbscan_labels==0].text[:5]

# Выглядит как внутренняя политика и оппозиция

In [ ]:
# label 1
df_clasters[df_clasters.dbscan_labels==1].text[:5]

# Узнала, что такое инаугурация. Вполне похоже на внешнюю политику

In [ ]:
# label 2
df_clasters[df_clasters.dbscan_labels==2].text[:5]

# Какие-то перемешанные темы внутренней и внешней политики

In [ ]:
# label 3
df_clasters[df_clasters.dbscan_labels==3].text[:5]

# Смерть дипломата тянет на мировые события или внешнюю политику

In [ ]:
# label 4
df_clasters[df_clasters.dbscan_labels==4].text[:5]

# Вот это точно мировые события

In [ ]:
# label 5
df_clasters[df_clasters.dbscan_labels==5].text[:5]

# А это точно теракты

In [ ]:
# label 6
df_clasters[df_clasters.dbscan_labels==6].text[:7]

# Криминальная хроника, тоже несложно

In [ ]:
# label 7
df_clasters[df_clasters.dbscan_labels==7].text[:5]

# Либо мировые события, либо технологии

In [ ]:
# label 8
df_clasters[df_clasters.dbscan_labels==8].text[:5]

# Точно спорт

In [ ]:
# label 9
df_clasters[df_clasters.dbscan_labels==9].text[:5]

# Определенно внутренняя политика

In [ ]:
# label 10
df_clasters[df_clasters.dbscan_labels==10].text[:5]

# Международные отношения или мировые события

In [ ]:
# label 11
df_clasters[df_clasters.dbscan_labels==11].text[:5]

# Проишествия  РФ

In [ ]:
# label 12
df_clasters[df_clasters.dbscan_labels==12].text[:5]

# Внутренняя политика

In [ ]:
# Оно даже интерпретируется
# По крайней мере, элементы внутри кластера похожи
# А вот с отделением кластеров друг от друга всё не очень

In [ ]:
# 3) Сравнение с правильными значениями

# Больше всего проблем с разграничением важных мировых событий и внутренней политики
# (Справедливости ради, я тоже их путаю)
# Зато неплохо справились с разделением более конкретных тем вроде террактов и культуры.

In [2]:
pd.crosstab(df.loc[:, 'class'], labels)

NameError: name 'pd' is not defined

In [3]:
from sklearn.metrics import adjusted_rand_score
adjusted_rand_score(true_label, labels)

NameError: name 'true_label' is not defined